In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.GAT_multihead import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/GAT/1k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/integrated_gat_hrnn', 'model': {'type': 'Integrated GAT-HRNN with Dual Attention', 'gat_units': 32, 'lstm_units': 128, 'num_lstm_layers': 4, 'attention_heads': 4, 'integration_units': 256}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5, 'optimizer': 'adam', 'loss': 'categorical_crossentropy'}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'num_agents': 2, 'num_roles': 2, 'path': 'games/tic-tac-toe/10k_single_agent.csv'}, 'input': [{'name': 'agent_features', 'shape': [2, 20]}, {'name': 'adjacency_matrix', 'shape': [2, 2]}, {'name': 'role_masks', 'shape': [2, 2]}], 'output': [{'name': 'event_prediction', 'num_classes': 9}, {'name': 'agent_prediction', 'num_classes': 2}, {'name': 'context_prediction', 'num_classes': 20}], 'evaluation': {'metrics': ['accuracy', 'precision', 'recall', 'f1_score']}, 'visualization': {'attention_maps': True, 'learning_curves': True, 'confusion_matrix': True}}


In [6]:
model_params = {
        'gat_units': config['model']['gat_units'],
        'lstm_units': config['model']['lstm_units'],
        'num_lstm_layers': config['model']['num_lstm_layers'],
        'attention_heads': config['model']['attention_heads'],
       'integration_units': config['model']['integration_units']
    }

In [7]:
# Load and preprocess data
df = load_data(f"../../../../data/processed/{config['data']['path']}")
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model
)


Epoch 1/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 100s 77ms/step - loss: 4.2312 - output_1_accuracy: 0.7951 - output_2_accuracy: 0.4244 - output_3_accuracy: 0.0936 - val_loss: 3.4977 - val_output_1_accuracy: 0.7951 - val_output_2_accuracy: 0.7510 - val_output_3_accuracy: 0.1486
Epoch 2/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 97s 80ms/step - loss: 2.7904 - output_1_accuracy: 0.8634 - output_2_accuracy: 0.8484 - output_3_accuracy: 0.2173 - val_loss: 2.4434 - val_output_1_accuracy: 0.9103 - val_output_2_accuracy: 0.8890 - val_output_3_accuracy: 0.2367
Epoch 3/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 99s 82ms/step - loss: 2.5271 - output_1_accuracy: 0.8935 - output_2_accuracy: 0.8798 - output_3_accuracy: 0.2331 - val_loss: 2.5876 - val_output_1_accuracy: 0.8777 - val_output_2_accuracy: 0.8680 - val_output_3_accuracy: 0.2186
Epoch 4/5
1207/1207 ━━━━━━━━━━━━━━━━━━━━ 99s 82ms/step - loss: 2.4409 - output_1_accuracy: 0.9089 - output_2_accuracy: 0.8883 - output_3_accuracy: 0.2432 - val_loss: 2.2824 - val_output_1_acc

Test Results:
loss: 2.2553
compile_metrics: 0.9357


ValueError: Unable to synchronously create dataset (name already exists)

In [ ]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

In [ ]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)